# Import libraries

In [19]:
# ! pip install faker 

In [20]:
import pandas as pd
import numpy as np
import random
from faker import Faker
from datetime import datetime, timedelta
from uuid import uuid4
from tqdm import tqdm
from random import randint, uniform
from faker.providers import BaseProvider

In [21]:
fake = Faker()
start_date = datetime.strptime("2005-01-01", "%Y-%m-%d")
end_date = datetime.strptime("2023-12-31", "%Y-%m-%d")

In [22]:
def generate_random_id():
    return(uuid4().hex)

# Data generation

## Customer data

In [23]:
from random import randint, uniform
from faker.providers import BaseProvider

class CameroonPhoneProvider(BaseProvider):
    def phone_number(self):
        """Generates a fake Cameroon phone number in a random format."""
        formats = [
            "+237 65ABCDEFG",
            "+237 67ABCDEFG",
            "+237 68ABCDEFG",
            "+237 69ABCDEFG",
        ]
        format_str = formats[randint(0, len(formats) - 1)]
        phone_number = format_str.replace("A", str(randint(0, 9)))
        phone_number = phone_number.replace("B", str(randint(0, 9)))
        phone_number = phone_number.replace("C", str(randint(0, 9)))
        phone_number = phone_number.replace("D", str(randint(0, 9)))
        phone_number = phone_number.replace("E", str(randint(0, 9)))
        phone_number = phone_number.replace("F", str(randint(0, 9)))
        phone_number = phone_number.replace("G", str(randint(0, 9)))
        return phone_number

fake = Faker()
fake.add_provider(CameroonPhoneProvider)

cam_phone_number = fake.phone_number()
print(cam_phone_number)

+237 652289546


In [24]:
customers_list = []
for i in tqdm(range(10000)):
    first_name = fake.first_name()
    last_name = fake.last_name()
    domain = fake.domain_name()
    email = f"{first_name}.{last_name}@{domain}"
    phone_number = cam_phone_number
    registration_date = fake.date_between(start_date, end_date)
    date_of_birth = fake.date_of_birth(minimum_age=30, maximum_age=75)
    address = fake.address()
    city = random.choice(["Yaoundé", "Douala", "Bafoussam", "Dschang", "Garoua", "Maroua", "Bamenda", "Ngaoundéré", "Nkongsamba", "Limbe", "Bertoua", "Mbouda", "Guider","Foumban", "Tibati", "Buea", "Kumba", "Kribi", "Loum", "Nkambe","Bangangté", "Baham", "Ndikiniméki", "Wum", "Batouri", "Yokadouma", "Mindif", "Meigang", "Abong-Mbang", "Bertoua", "Bogo", "Dir", "Poli", "Mbouda", "Bakoum", "Ngaoundal", "Figuil", "Mayo-Darlé", "Mayo-Rey", "Betaré-Oya", "Guidiguis", "Minjibir", "Mbankomo", "Ntui", "Mengang", "Akonolinga", "Ebolowa", "Abé", "Sangmélima", "Oveng", "Bipindi", "Meyomessé",])
    region = random.choice(["Nord", "Sud", "Est", "Ouest", "Centre", "Littoral", "Adamaoua", "Nord-ouest", "Sud-ouest", "Extrème-nord"])

    date_dict = {
        'customer_id' :generate_random_id(), 
        'first_name': first_name, 
        'last_name': last_name,
        'email': email,
        'phone_number': phone_number,
        'registration_date': registration_date,
        'date_of_birth' : date_of_birth,
        'address': address,
        'city': city,
        'region' : region
        }
    customers_list.append(date_dict)
customer_df = pd.DataFrame(customers_list)
print(customer_df.shape)
customer_df.head()

(10000, 10)


,customer_id,first_name,last_name,email,phone_number,registration_date,date_of_birth,address,city,region
0,0b57f7920e2b4ec695d20b5a1105fe1b,Amy,Morton,Amy.Morton@lindsey-bradshaw.info,+237 652289546,2023-06-05,1953-08-06,75420 Castaneda Highway Suite 100\nEast Jordan...,Foumban,Littoral
1,84f19f2cb3f54bd0a52e7b50a25c2758,Michael,Solomon,Michael.Solomon@brady-scott.com,+237 652289546,2007-04-03,1986-04-14,"343 Elizabeth Curve\nLake Amybury, AK 25564",Yaoundé,Centre
2,65cf45e3c8894c0ba5757a7d7eb65d1b,David,Strong,David.Strong@hill.com,+237 652289546,2016-02-21,1981-12-13,"8875 Joseph Camp\nNew Johnhaven, AL 32679",Sangmélima,Centre
3,221fcf1ad35843e18c7a956a71661eaa,Glenda,Phillips,Glenda.Phillips@powell.org,+237 652289546,2010-03-31,1963-03-07,"754 Gregory Unions Apt. 116\nTranburgh, WI 45075",Dir,Sud-ouest
4,71f7ec505b3d460f8ef9a1ffcaaf5827,Stephanie,Beck,Stephanie.Beck@adams-guzman.info,+237 652289546,2020-11-04,1961-12-24,"14677 Eric Pass\nCherylshire, IN 88302",Mayo-Darlé,Centre


In [25]:
customer_df.to_csv('../Data/Train/customers.csv')

## Device data

In [26]:

#creation de id

def create_id():
    return str(uuid4())

#creation des types d'appareils
def generate_device_type():
    return random.choice(['mobile', 'tablette'])

#creation des os

def generate_os():
    return random.choice(['android', 'ios', 'mac', 'windows', 'linux'])

#creation des fabricants
manufacturer_and_model={
                        'dell':['XPS 13', 'INSPIRON 15', 'LATITUDE 7410', 'PRECISION 5550'],
                        'HP': ['spectre x360', 'envy 13', 'pavillon 15', 'OMEN 15'],
                        'LG': ['spectre x3670', 'envy45','Gram 17', 'Wing 5G', ' NanoCell TV'],
                        'SONY':['Xperia 1 III', 'Xperia 5 II', 'Bravia OLDE', 'VAIO SX14'],
                        'Apple':['iphone 13', 'ipad pro', 'iphone Xr'],
                        'Sharp':['Aquos R6', 'aquos sense 5g', ' dynabook v8']
                        }
#def generate_manufacturer():
    #return random.choices(['dell', 'hp', 'lg', 'sony', 'apple', 'sharp', 'lenovo', 'acer', 'tecno','itel', 'huawei', 'samsung', 'google'])

#creation des modeles

def generate_model(type):
    return random.choice(manufacturer_and_model[type])

#creation de la table devices

def create_devices_table(lenght= 100000):

    devices= pd.DataFrame()

    # les listes
    device_id_list=[]
    devices_type_list=[]
    devices_manufacturer_list=[]
    devices_model_list=[]
    devices_last_used_list=[]
    devices_os_list=[]

    #generer les valeurs
    for i in tqdm(range(lenght)):
        devices_id=create_id()
        devices_type= generate_device_type()
        devices_last_used= fake.date_time_between(start_date= '-5y', end_date='now')
        devices_os= generate_os()
        devices_manufacturer= random.choice(list(manufacturer_and_model.keys()))
        devices_model= generate_model(devices_manufacturer)

        #ajout dans les listes
        device_id_list.append(devices_id)
        devices_type_list.append(devices_type)
        devices_model_list.append( devices_model)
        devices_manufacturer_list.append( devices_manufacturer)
        devices_last_used_list.append(devices_last_used)
        devices_os_list.append(devices_os)
        
     #assigner aux colonnes de la table devices
    devices['device_id']= device_id_list
    devices['device_type']= devices_type_list
    devices['os']= devices_os_list
    devices['manufacturer']= devices_manufacturer_list
    devices['model']= devices_model_list
    devices['last_used']= devices_last_used_list

    return devices

devices= create_devices_table()
devices

100%|██████████| 100000/100000 [00:09<00:00, 10182.27it/s]


,device_id,device_type,os,manufacturer,model,last_used
0,fa09a737-b144-421a-a422-9eb590e96f3c,mobile,windows,dell,PRECISION 5550,2019-12-16 16:00:26
1,46f8e813-8811-4ec4-bca2-dc31325182c3,mobile,linux,SONY,Xperia 1 III,2021-06-07 02:54:20
2,3d059cec-1133-42a2-8489-fab1cdcb6b57,mobile,mac,LG,NanoCell TV,2021-04-03 12:09:10
3,43c60112-08d1-477e-ac67-82f2fcdd1784,tablette,ios,SONY,Xperia 1 III,2023-03-04 03:32:42
4,ab6eeac1-2759-49cb-b58e-8d697911f565,mobile,ios,SONY,VAIO SX14,2021-03-24 20:03:47
...,...,...,...,...,...,...
99995,a1dbbf67-4a74-4dd0-a013-02f42d6d171b,tablette,windows,HP,spectre x360,2024-03-22 23:29:59
99996,fb827566-c21a-481e-a49f-8d87672cc007,mobile,windows,LG,spectre x3670,2022-06-24 10:48:34
99997,e9696737-6453-4ccf-a73f-6e002b68cdb6,tablette,ios,Apple,iphone 13,2019-10-13 04:49:00
99998,1f4ba4d6-932a-435c-a1cc-76eee2f5cc08,mobile,mac,dell,PRECISION 5550,2019-10-05 08:50:30


In [27]:
devices.to_csv('../Data/Train/devices.csv')

## Transaction data

In [28]:
transaction_list=[]
customer_id_list = customer_df["customer_id"]

for i in tqdm(range(100000)):
    transaction_id = generate_random_id()
    customer_id = random.choice(customer_id_list)
    transaction_type = random.choice(["retrait", "depôt", "transfert"])
    amount = uniform(100, 10000000)
    transaction_date = fake.date_between(start_date, end_date)
    location = fake.address()
    device_id = random.choice(devices["device_id"])
    status = random.choice(["échouée", "réussie"])
    is_fraud = random.choice([0,1])

    transaction_dict = {
        "transaction_id": transaction_id,
        "customer_id": customer_id,
        "transaction_type": transaction_type,
        "amount": amount,
        "transaction_date": transaction_date,
        "location": location,
        "device_id": device_id,
        "status": status,
        "is_fraud": is_fraud
    }

    transaction_list.append(transaction_dict)

transaction_df = pd.DataFrame(transaction_list)
print(transaction_df.shape)
transaction_df.head()

(100000, 9)


,transaction_id,customer_id,transaction_type,amount,transaction_date,location,device_id,status,is_fraud
0,2771e95a45564c57a443f987ef296ec2,ba7291aefb8442ae8fbf24600f826965,depôt,4.938843e+06,2018-06-06,"742 Kayla Gardens\nEricbury, NY 87241",0f455c1f-7583-47a4-8328-bcf5eea6a3a6,échouée,0
1,49eee058f8fa4b09b07e36cc6d9b4f72,ee42efad22eb477fa01d7fcef21a0150,transfert,7.131117e+05,2021-03-31,"7989 Henson Plaza Apt. 628\nFarmerton, WA 07218",1ff3ad61-1c14-4d26-951d-9ab17f61d2b1,réussie,0
2,f4ff1dff7343499181f1f1056b3c7281,03aec76a798f4a239add8319dae80ccc,depôt,7.846708e+06,2020-06-15,Unit 7694 Box 0741\nDPO AA 34017,a2bc3048-7f09-4ea3-b494-b003805dc8aa,échouée,0
3,d391763f90dd4016a72cc4b353daae46,29faf67f28d84c5296b45789205cf7e7,retrait,9.654389e+06,2021-01-26,"24172 Benson Flats Apt. 559\nFrankmouth, NE 35774",0026493d-23b0-4a4e-9670-5aac6b74ab40,échouée,0
4,c884de29c99049d28727f74e755b5aa6,a91ea16d5c8b4e40bc76825e626dfdc0,transfert,6.063242e+05,2006-05-01,"001 David Canyon\nLake Ellen, ND 05283",dff7fa60-8e05-41c8-9a1c-b34235747e5d,échouée,1


In [29]:
transaction_df.to_csv('../Data/Train/transactions.csv')

## Transaction history data

In [30]:
transaction_history_list = []

for i, transaction in transaction_df.iterrows():
    history_id = generate_random_id()
    customer_id = transaction.customer_id
    transaction_id = transaction.transaction_id
    previous_balance = transaction.amount + uniform(0, 10000000)

    if transaction.transaction_type == "depôt":
        new_balance = previous_balance + transaction.amount
    else:
        new_balance = previous_balance - transaction.amount
        
    transaction_date = transaction.transaction_date

    transaction_history_dict = {
        "history_id" : history_id,
        "customer_id" : customer_id,
        "transaction_id" : transaction_id,
        "previous_balance" : previous_balance,
        "new_balance" : new_balance,
        "transaction_date": transaction_date
    }
    transaction_history_list.append(transaction_history_dict)

transaction_history_df = pd.DataFrame(transaction_history_list)
print(transaction_history_df.shape)
transaction_history_df.head()

(100000, 6)


,history_id,customer_id,transaction_id,previous_balance,new_balance,transaction_date
0,c93d5622d04246e4a27280f6e28803b9,ba7291aefb8442ae8fbf24600f826965,2771e95a45564c57a443f987ef296ec2,1.085595e+07,1.579479e+07,2018-06-06
1,3a4610992ea54f2f8259dd1a4d72a86f,ee42efad22eb477fa01d7fcef21a0150,49eee058f8fa4b09b07e36cc6d9b4f72,8.074497e+06,7.361385e+06,2021-03-31
2,f90503a9086741aa8647b316f4c765ac,03aec76a798f4a239add8319dae80ccc,f4ff1dff7343499181f1f1056b3c7281,1.546046e+07,2.330717e+07,2020-06-15
3,61e351535a71430fada420c196a678be,29faf67f28d84c5296b45789205cf7e7,d391763f90dd4016a72cc4b353daae46,1.134622e+07,1.691835e+06,2021-01-26
4,0f71e744641641a88d9edea6aa30612b,a91ea16d5c8b4e40bc76825e626dfdc0,c884de29c99049d28727f74e755b5aa6,2.283778e+06,1.677454e+06,2006-05-01


In [31]:
transaction_history_df.to_csv('../Data/Train/transaction_history.csv')

## Transaction pattern data

In [32]:
customer_id_list = customer_df["customer_id"]
print(customer_id_list)

0       0b57f7920e2b4ec695d20b5a1105fe1b
1       84f19f2cb3f54bd0a52e7b50a25c2758
2       65cf45e3c8894c0ba5757a7d7eb65d1b
3       221fcf1ad35843e18c7a956a71661eaa
4       71f7ec505b3d460f8ef9a1ffcaaf5827
                      ...               
9995    02a3230491c54603aaf67a726237b87d
9996    b02a2a9717d4437a8ade9bf203d247ac
9997    49897879817540f9b37631530179c0e4
9998    da9aa8a2741242c5a5e030fdc4eeb04d
9999    5bed1e8f311e4610a7836b3cbfa61c6a
Name: customer_id, Length: 10000, dtype: object


In [33]:
import numpy as np
import pandas as  pd 
import random
from faker import Faker
from uuid import uuid4
from tqdm import tqdm

faker= Faker()

# Fonction pour générer un identifiant unique
def create_id():
    return str(uuid4())

# Fonction pour générer un montant moyen aléatoire
def generate_average_amount():
    return round(random.uniform(5.0, 1000.0), 2)

# Fonction pour générer une fréquence de transactions aléatoire
def generate_frequency():
    return random.randint(1, 30)

# Fonction pour générer une localisation fréquente aléatoire
def generate_most_common_location():
    return faker.city()

# Fonction pour générer un appareil le plus utilisé aléatoire
def generate_most_common_device():
    return random.choice(['mobile', 'tablette'])

# Fonction pour créer une table de transaction patterns
def create_transaction_patterns_table(length=100000):

    transaction_patterns = pd.DataFrame()

    # Les listes
    pattern_id_list = []
    customer_id_list = customer_df["customer_id"].tolist()
    average_amount_list = []
    frequency_list = []
    most_common_location_list = []
    most_common_device_list = []
    foreign_key_list=[]  # liste des cles etrangeres qui seront aleatoirement choisies

    # Générer les valeurs
    for i in tqdm(range(length)):
        pattern_id = create_id()
        customer_id = random.choice(customer_id_list) 
        average_amount = generate_average_amount()
        frequency = generate_frequency()
        most_common_location = generate_most_common_location()
        most_common_device = generate_most_common_device()

        # Ajouter dans les listes
        pattern_id_list.append(pattern_id)
        foreign_key_list.append(customer_id)
        average_amount_list.append(average_amount)
        frequency_list.append(frequency)
        most_common_location_list.append(most_common_location)
        most_common_device_list.append(most_common_device)
    
    # Assigner aux colonnes de la table transaction patterns (en dehors de la boucle)
    transaction_patterns['pattern_id'] = pattern_id_list
    transaction_patterns['customer_id'] = foreign_key_list
    transaction_patterns['average_amount'] = average_amount_list
    transaction_patterns['frequency'] = frequency_list
    transaction_patterns['most_common_location'] = most_common_location_list
    transaction_patterns['most_common_device'] = most_common_device_list

    return transaction_patterns

# Créer une table de transaction patterns
transaction_patterns = create_transaction_patterns_table()

# Afficher les premières lignes de la table transaction patterns
transaction_patterns


100%|██████████| 100000/100000 [00:39<00:00, 2531.42it/s]


,pattern_id,customer_id,average_amount,frequency,most_common_location,most_common_device
0,0b9e4a67-e89c-436b-8297-38c4a50e47a3,53b9bba3106a442aa105d763a5e9eaff,370.77,20,Jeremyland,mobile
1,119e294d-78c8-4472-903b-ec2e3333fe5b,4d74f37545ea48a6ad9cfa01a45604c1,951.33,15,Mathewstad,tablette
2,d1f04bae-4e2f-4416-848d-2fb7cf768877,752b58aec2cf4d48a38af0bce96d93b4,75.03,10,Johnsonport,tablette
3,7b8ee383-1625-462c-ad6e-3244f80f314a,dfa5307e453e41b0ad3b5def0464ab0a,759.98,21,Jessicaland,tablette
4,8c68e3c9-6a71-4453-b68c-b380e3f60107,403a5f24c30f4cf38bea89072ccb2a36,756.88,8,Ivanville,mobile
...,...,...,...,...,...,...
99995,31af8c84-2d3c-4d75-a19c-82409138cbfb,856c672f10c84700a80180b68730e9e7,992.49,2,Port Jeremyland,tablette
99996,e050c6f2-8166-483f-9e57-54bfe7f852d6,b5a028b726eb4660ac22621ba4ad659f,207.95,5,Anthonyland,tablette
99997,45972d61-3293-49fa-9a99-2c7a78179388,6fde244ea15a470090e1115e63c6c920,980.51,4,New Brandontown,mobile
99998,06a9245e-e486-4a1d-a30a-b05611f86938,8dc22779b08c477a9dfb1c8d8b8724db,216.81,13,East Ryan,tablette


In [34]:
transaction_patterns.to_csv('../Data/Train/transaction_patterns.csv')

## alerts data

In [35]:
# Fonction pour générer un identifiant unique
def create_id():
    return str(uuid4())

# Fonction pour générer une date et heure aléatoire
def generate_alert_date():
    return faker.date_time_between(start_date='-5y', end_date='now')

# Fonction pour générer un type d'alerte aléatoire
def generate_alert_type():
    return random.choice(['suspicious activity', 'large transaction'])

# Fonction pour générer un statut d'alerte aléatoire
def generate_alert_status():
    return random.choice(['open', 'closed'])

# Fonction pour générer une description d'alerte aléatoire
def generate_description():
    return faker.text(max_nb_chars=250)

# Fonction pour créer une table d'alertes
def create_alerts_table(length=100):

    alerts = pd.DataFrame()

    # Les listes
    alert_id_list = []
    transaction_id_list = transaction_df["customer_id"].tolist()
    alert_date_list = []
    alert_type_list = []
    alert_status_list = []
    description_list = []
    foreign_key_alerts_list=[]

    # Générer les valeurs
    for i in tqdm(range(length)):
        alert_id = create_id()
        transaction_id = random.choice(transaction_id_list) # Remplacer par un identifiant de transaction réel si disponible
        alert_date = generate_alert_date()
        alert_type = generate_alert_type()
        alert_status = generate_alert_status()
        description = generate_description()

        # Ajouter dans les listes
        alert_id_list.append(alert_id)
        foreign_key_alerts_list.append(transaction_id)
        alert_date_list.append(alert_date)
        alert_type_list.append(alert_type)
        alert_status_list.append(alert_status)
        description_list.append(description)
    
    # Assigner aux colonnes de la table alerts (en dehors de la boucle)
    alerts['alert_id'] = alert_id_list
    alerts['transaction_id'] = foreign_key_alerts_list
    alerts['alert_date'] = alert_date_list
    alerts['alert_type'] = alert_type_list
    alerts['alert_status'] = alert_status_list
    alerts['description'] = description_list

    return alerts

# Créer une table d'alertes
alerts = create_alerts_table()

# Afficher les premières lignes de la table alerts
alerts

100%|██████████| 100/100 [00:00<00:00, 498.27it/s]


,alert_id,transaction_id,alert_date,alert_type,alert_status,description
0,dc948a5b-9495-46e4-9ccf-d700651c0508,545d143c0f3c487eb3799d002ab63f8d,2021-01-04 00:54:20,suspicious activity,closed,Political expert figure same wide dinner. Crim...
1,7d2a75aa-2ec6-4cc1-8c8e-87f1529a7a84,7850b6d7ddca4b0cb167ca876aa563f9,2021-01-24 01:18:46,large transaction,open,Way market forget beat. Become interesting din...
2,307f9886-9b0e-407b-a3ce-b7abd1331b66,9b1a2550e14543078ae20f40b7382b44,2020-12-06 17:22:28,large transaction,closed,Shoulder color society scene rock news. Popula...
3,d284fbe2-e3d0-44e5-af9c-be6701c60df2,42759a74bfe24059beadb7bae6c3af91,2019-11-25 16:17:27,large transaction,open,Hotel research activity data hear institution ...
4,00687cba-0aec-4b8c-9a73-2bbedfb0e2e9,90333a909dcd4198afe6101b24b5e6d4,2024-02-22 13:02:29,suspicious activity,closed,Social pattern purpose take recently against s...
...,...,...,...,...,...,...
95,133a195a-824c-4704-9e45-b512ce069280,8b556f5459d641fdaeb26da8424b906c,2020-01-14 23:38:18,suspicious activity,open,Give safe form per.\nReport process whole matt...
96,4596acca-9d95-4330-ab68-999a60e5a90b,cfe5bf1c1ec34b1c95c9d020a5679629,2023-06-25 17:14:43,large transaction,open,Early personal Democrat writer shoulder patter...
97,d09080e9-ea3e-4f38-9ee5-de41918207c3,d0e677002b1848f388f8ba3061b16c68,2021-09-01 02:37:30,suspicious activity,open,Whose apply attorney offer source. Raise likel...
98,15e89e3e-ed38-4cbf-9945-357e82cdf9dd,742f390a8eaa4407b5eef3a2d31ff0b3,2019-10-02 02:31:15,suspicious activity,closed,Rest education kitchen establish direction. Ab...


In [36]:
alerts.to_csv('../Data/Train/alerts.csv')

## fraude_cases data 

## regions data

In [4]:
from glob import glob

list_data = glob('../Data/Train/*')
print(list_data)


['../Data/Train\\alerts.csv', '../Data/Train\\customers.csv', '../Data/Train\\devices.csv', '../Data/Train\\transactions.csv', '../Data/Train\\transaction_history.csv', '../Data/Train\\transaction_patterns.csv']


In [5]:
import os 
os.sep

'\\'

In [10]:
list_data[0].split(os.sep)[-1].split('.')[0]

'alerts'